In [28]:
import numpy as np
import pandas as pd
import nltk

In [29]:
data=pd.read_csv('./相关资料/Reviews.csv')

# 选取good bad 评论

In [30]:
class get_good_bad_data(object):
    '''
    分数为5为good
    分数为1为bad
    '''
    def __init__(self,data):
        self.data=data
        
    def get_two_groups(self):
        '''
        选取分数最高，而且赞同率100%的样本为正样本
        选取分数最低，而且赞同率100%的样本为负样本
        '''
        opts=np.array(self.data.loc[0:,['HelpfulnessNumerator','HelpfulnessDenominator','Score']])
        opts[:,0:-1]+=1
        index_good=np.where((opts[:,-1]==5) * (opts[:,0]/opts[:,1]==1)*(opts[:,0]!=1 )* (opts[:,1]!=1))
        index_bad=np.where((opts[:,-1]==1) * (opts[:,0]/opts[:,1]==1) * (opts[:,0]!=1 )* (opts[:,1]!=1))

        return index_bad,index_good
    
    def get_two_summary(self):
        
        index_bad,index_good=self.get_two_groups()
        index_bad,index_good=index_bad[0],index_good[0]
        num=len(np.array(self.data))
        index_other=np.array(range(num))
   
        inds=np.concatenate((index_good,index_bad))
        index_other=np.delete(index_other,inds)
        
        
        
        good_summary=np.array(self.data.loc[0:,['Summary']])[index_good]
        bad_summary=np.array(self.data.loc[0:,['Summary']])[index_bad]
        
        test_summary=np.array(self.data.loc[0:,['Summary']])[index_other]
        
        scores=np.array(self.data.loc[0:,['Score']])[index_other]
        
        return good_summary,index_good,bad_summary,index_bad,scores,test_summary,index_other
    
    
        

In [31]:
good_summary1,index_good,bad_summary1,index_bad,scores,test_summary,index_other=get_good_bad_data(data).get_two_summary()

# 生成训练集，测试集，和标签

# 训练集正样本1万，负样本8千，测试集正样本3000，负样本2000

In [32]:
all_summary=np.concatenate((good_summary1,bad_summary1),axis=0)
good_summary=good_summary1[0:10000]
bad_summary=bad_summary1[0:8000]
test_=np.concatenate((good_summary1[10000:13000],bad_summary1[8000:10000]))
labels=np.zeros(len(test_))
labels[0:3000]+=1

# 伯努利朴素贝叶斯

In [33]:
class NaiveBayes(object):
    
    def __init__(self,good_summary,bad_summary):
        
        self.sen_prod_good=None
        self.sen_prod_bad=None
        
        self.prod_good=None
        self.prod_bad=None
        
        self.good_summary=good_summary
        self.bad_summary=bad_summary
        
    def train(self):
        good_num=len(self.good_summary)
        all_summary=np.concatenate((self.good_summary,self.bad_summary),axis=0)
        labels=np.zeros(len(all_summary))
        labels[0:good_num]+=1
        
        self.prod_good=good_num/len(all_summary)
        self.prod_bad=1-self.prod_good
        
        words=self.get_word_vec()
        
        self.sen_prod_good=np.zeros(len(words))+0.001
        self.sen_prod_bad=np.zeros(len(words))+0.001
        sum1=0
        sum0=0
        
        for i in range(len(all_summary)):  
            if labels[i]==1:
                vec=self.summary_to_vec(nltk.word_tokenize(all_summary[i][0]),words)
                self.sen_prod_good+=vec
                sum1+=np.sum(vec)
            if labels[i]==0:
                vec=self.summary_to_vec(nltk.word_tokenize(all_summary[i][0]),words)
                self.sen_prod_bad+=vec
                sum0+=np.sum(vec)
                
        
        self.sen_prod_bad=np.log(self.sen_prod_bad/sum0)
        self.sen_prod_good=np.log(self.sen_prod_good/sum1)
        
    def predict(self, others):
        
        self.train()
        
        words=self.get_word_vec()
        
        all_num=len(others)
        pre_list=[]
        for i in range(all_num):
            vec=self.summary_to_vec(nltk.word_tokenize(others[i][0]),words)
           
            p_good=np.sum(self.sen_prod_good*vec)+np.log(self.prod_good)
            p_bad=np.sum(self.sen_prod_bad*vec)+np.log(self.prod_bad)
            
            if p_bad>=p_good:
                pre_list.append(0)
            if p_bad<p_good:
                pre_list.append(1)
                
        return np.array(pre_list)
        
                
        
    def get_word_vec(self):
        all_summary=np.concatenate((self.good_summary,self.bad_summary),axis=0)
        list_words=[]
        for i in range(len(all_summary)):
            list_words=list_words+nltk.word_tokenize(all_summary[i][0])
        words=list(set(list_words))
        return words
    
    def summary_to_vec(self,summary,words):
        
        '''
        summary:列表
        '''
        
        vec=np.zeros(len(words))
        
        for word in summary:
            
            if  word in words:
                indexs=words.index(word)
                if vec[indexs]==0:
                    vec[indexs]+=1
        
        return vec
    
        
        
            

# 正确率预测

In [34]:
pre_=NaiveBayes(good_summary,bad_summary).predict(test_)

In [35]:
print(sum(pre_==labels)/len(labels))

0.8778
